## インストール(初回のみ)
事前に
- pyenv をインストール
- `pyenv install 3.8.13` を実行

しておく

In [ ]:
!sh install.sh

---

In [ ]:
# 仮想環境ディレクトリのパッケージを使うため
import site
site.addsitedir('env/lib/python3.8/site-packages')

In [ ]:
import torch
import matplotlib.pyplot as plt
from IPython.display import Audio

## IO

In [ ]:
from utils.audio.wave import load_wave, metadata_wave
from utils.audio.plot import show_spec, show_wave

In [ ]:
def show(znt, fs):
    display(Audio(data=znt, rate=fs))

    fig, axes = plt.subplots(2, 1, figsize=(16, 4), tight_layout=True)

    show_wave(znt, fs, ax=axes[0], color='b')

    axes[0].tick_params(axis='x', bottom=False, labelbottom=False)
    axes[0].set_xlim([0, znt.shape[-1]/fs])
    axes[0].set_xlabel('')

    show_spec(znt, fs, stft_dict['n_fft'], ax=axes[1])

    plt.show()

In [ ]:
filename = 'data/trim.wav'
data, fs = load_wave(filename)

# show(data, fs)

xnt = data[:, fs*10:fs*20]

## BSS
分離パラメータの詳細は pyroomacoustics から確認
> https://pyroomacoustics.readthedocs.io/en/pypi-release/pyroomacoustics.bss.html
- AuxIVA
- FastMNMF
- ILRMA

以下，使用は非推奨

自己実装
- ~~NMFLGM~~

GPUテンソルでのデバッグがまだ(CPU上では動く)
> https://github.com/onolab-tmu/overiva
- ~~OverIVA~~
- ~~OGIVE~~

In [ ]:
from utils.audio.bss.auxiva import AuxIVA
from utils.audio.bss.fastmnmf import FastMNMF
from utils.audio.bss.ilrma import ILRMA

# from utils.audio.bss.lgm import NMFLGM
# from utils.audio.bss.overiva import OverIVA
# from utils.audio.bss.ogive import OGIVE

In [ ]:
stft_dict = dict(n_fft=1024, hop_length=256, window=torch.hann_window)
device = 'cpu'
model = AuxIVA(**stft_dict).to(device)

In [ ]:
ynt = model(xnt.to(device), n_iter=100)
_ = [show(yt, fs) for yt in ynt]